In [342]:
import pandas as pd
import numpy as np 
data_train = pd.read_csv("data_golf_train.csv")
data_train.head()


,Outlook,Temperature,Humidity,Windy,Play
0,Sunny,30.5,90,False,No
1,Sunny,29.0,82,True,No
2,Overcast,28.2,95,False,Yes
3,Rain,21.0,86,False,Yes
4,Rain,5.3,60,False,Yes


In [345]:
# guardamos las probabilidades para cada combinación P(x_j|c_i)
P = {} # se puede mejorar formato
P["Play"] = data_train["Play"].value_counts()/data_train["Play"].shape

# vemos las clases que tenemos
for c_i in data_train["Play"].unique():
    # condicionamos la base a las observaciones de la clase a evaluar
    data_cond = data_train[data_train["Play"]==c_i]
    # revisamos los atributos restante para obtener su distribución
    for x_j in data_cond.drop(["Play"],axis=1):
        # si es categórica vemos la fecuencia de cada valor
        if data_train[x_j].dtypes == object or data_train[x_j].dtypes == bool:
            P[x_j,c_i] = data_cond[x_j].value_counts()/data_cond[x_j].shape
        # si es numérica vemos los parámetros de la normal
        if data_train[x_j].dtypes == "int64" or data_train[x_j].dtypes == float:
            P[x_j,c_i] = np.mean(data_cond[x_j]),np.std(data_cond[x_j])


In [347]:
data_test = pd.read_csv("data_golf_test.csv")
data_test.head()

,Outlook,Temperature,Humidity,Windy
0,Sunny,20,50,False
1,Sunny,30,50,True
2,Rain,10,60,False
3,Rain,20,60,True


In [348]:
def pdf_norm(u,sigma,x):
    return np.exp((-(x-u)**2)/(2*sigma**2))/(sigma*(2*np.pi))

In [362]:

# Predicciones
def naive_bayes(df):
    # argmax de map
    K = []

    for index, row in df.iterrows():
        # guardamos los valores del map (creo que es el map)
        MAP_j = [[],[]]
        # vemos las clases que podemos predecir
        for c_i in data_train["Play"].unique():
            # vemos el map que obtendriamos en cada clase
            map_i = P["Play"][c_i] # partimos con el término P(c_i)
            for x_j in df.columns:
                # vamos multiplicado los P(x_j|c_i) según corresponda

                #categórica
                if type(row[x_j]) == str or type(row[x_j]) == bool:
                    if row[x_j] in P[x_j,c_i]:
                        map_i *= P[x_j,c_i][row[x_j]]
                    # si no existe la combinación p=0
                    else: 
                        map_i = 0     
                #numérica
                if type(row[x_j]) == int or type(row[x_j]) == float:
                    map_i *= pdf_norm(P[x_j,c_i][0],P[x_j,c_i][1],row[x_j])

            # guardamos: 
            MAP_j[0].append(map_i) # map de la clase c_i
            MAP_j[1].append(c_i) # clase c_i
        # elegimos la clase que maximiza la función MAP
        K.append(MAP_j[1][np.argmax(MAP_j[0])])
    return K


In [366]:
naive_bayes(data_test)
#naive_bayes(data_train.drop(["Play"],axis=1))

['Yes', 'Yes', 'Yes', 'Yes']

In [358]:
from sklearn.linear_model import LogisticRegression
#expresamos las variables categóricas de forma numérica

#hacemos dummies de Outlook, Windy y Play
data_train_num = pd.get_dummies(data_train,columns=["Outlook","Windy","Play"])
data_test_num = pd.get_dummies(data_test,columns=["Outlook","Windy"])


# sacamos una columna para cada categoría, cosa que no hayan problemas de dependecia lineal
# notar que data_test no tiene ningún valor de Overcast, por lo que no es necesario sacar la columna ya que no fue creada
data_train_num = data_train_num.drop(["Outlook_Overcast","Windy_False","Play_No"],axis=1)
data_test_num = data_test_num.drop(["Windy_False"],axis=1)


In [367]:
reg = LogisticRegression().fit(data_train_num.drop(["Play_Yes"],axis=1), data_train_num["Play_Yes"])
reg.predict(data_test_num)

array([1, 1, 1, 1], dtype=uint8)